In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

data = pd.read_csv("E:\Final year project\iot23_combined.csv",nrows = 100000)


In [12]:

label_mapping = {
    'PartOfAHorizontalPortScan': 0,
    'Benign': 1,
    'Okiru': 2,
    'DDoS': 3,
    'C&C': 4,
    'Attack': 5,
    'C&C-HeartBeat': 6,
    'C&C-FileDownload': 7,
    'C&C-Torii': 8,
    'FileDownload': 9,
    'C&C-HeartBeat-FileDownload': 10,
    'Okiru-Attack': 11,
    'C&C-Mirai': 12,
    '-   benign   -': 13,
}


data['label'] = data['label'].map(label_mapping)

In [13]:

non_binary_feature_columns = [
    'duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes',
    'resp_pkts', 'resp_ip_bytes'
]

scaler = StandardScaler()

data[non_binary_feature_columns] = scaler.fit_transform(data[non_binary_feature_columns])


In [14]:
binary_feature_columns = [
    'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ',
    'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH',
    'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3',
    'conn_state_SF', 'conn_state_SH', 'conn_state_SHR'
]

label_encoder = LabelEncoder()
for column in binary_feature_columns:
    data[column] = label_encoder.fit_transform(data[column])

In [15]:
data

,Unnamed: 0,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0,0.901146,-0.128871,-0.078198,0.0,0.520869,0.693888,-0.098711,-0.098362,0,...,0,0,0,1,0,0,0,0,0,0
1,1,-0.299475,-0.128871,-0.078198,0.0,-0.240652,-0.214082,-0.098711,-0.098362,0,...,0,0,0,1,0,0,0,0,0,0
2,2,-0.299475,-0.128871,-0.078198,0.0,-0.240652,-0.214082,-0.098711,-0.098362,0,...,0,0,0,1,0,0,0,0,0,0
3,3,0.901149,-0.128871,-0.078198,0.0,0.520869,0.693888,-0.098711,-0.098362,1,...,0,0,0,1,0,0,0,0,0,0
4,4,-0.299475,-0.128871,-0.078198,0.0,-0.240652,-0.214082,-0.098711,-0.098362,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,-0.299475,-0.128871,-0.078198,0.0,-0.240652,-0.214082,-0.098711,-0.098362,1,...,0,0,0,1,0,0,0,0,0,0
99996,99996,-0.299475,-0.128871,-0.078198,0.0,-0.240652,-0.214082,-0.098711,-0.098362,0,...,0,0,0,1,0,0,0,0,0,0
99997,99997,-0.299475,-0.128871,-0.078198,0.0,-0.240652,-0.365410,-0.098711,-0.098362,1,...,0,0,0,1,0,0,0,0,0,0
99998,99998,0.901249,-0.128871,-0.078198,0.0,0.520869,0.693888,-0.098711,-0.098362,0,...,0,0,0,1,0,0,0,0,0,0


In [16]:
X = data[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
y = data['label']

In [17]:
y.unique()

array([0, 1], dtype=int64)

In [18]:
# Convert Pandas DataFrames to NumPy arrays
X = X.values
y = y.values

# Reshape input for channels dimension
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode target labels as categorical
num_classes = 14
y = to_categorical(y, num_classes=num_classes)

In [19]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)


In [21]:
model = Sequential()

# Convolutional layers
model.add(Conv1D(filters=36, kernel_size=5, activation='relu', input_shape=(24, 1)))
model.add(BatchNormalization())

model.add(Conv1D(filters=72, kernel_size=5, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=144, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=288, kernel_size=3, activation='relu'))
model.add(BatchNormalization())


model.add(GlobalAveragePooling1D())
model.add(Dense(num_classes, activation='softmax'))


optimizer = Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=10)

Epoch 1/10
2188/2188 [==============================] - 31s 12ms/step - loss: 0.1995 - accuracy: 0.9545 - val_loss: 0.1891 - val_accuracy: 0.9507
Epoch 2/10
2188/2188 [==============================] - 24s 11ms/step - loss: 0.1677 - accuracy: 0.9559 - val_loss: 0.1740 - val_accuracy: 0.9551
Epoch 3/10
2188/2188 [==============================] - 23s 11ms/step - loss: 0.1678 - accuracy: 0.9560 - val_loss: 0.1876 - val_accuracy: 0.9551
Epoch 4/10
2188/2188 [==============================] - 22s 10ms/step - loss: 0.1657 - accuracy: 0.9560 - val_loss: 0.2219 - val_accuracy: 0.9551
Epoch 5/10
2188/2188 [==============================] - 22s 10ms/step - loss: 0.1643 - accuracy: 0.9560 - val_loss: 0.2422 - val_accuracy: 0.9545
Epoch 6/10
2188/2188 [==============================] - 22s 10ms/step - loss: 0.1638 - accuracy: 0.9560 - val_loss: 0.1954 - val_accuracy: 0.9550
Epoch 7/10
2188/2188 [==============================] - 22s 10ms/step - loss: 0.1628 - accuracy: 0.9559 - val_loss: 0.2470 -